# 04 - About $\textsf{RGSW}$
---

From <a href='https://eprint.iacr.org/2020/086'> Bootstrapping in FHEW-like Cryptosystems </a>

In [1]:
import numpy as np
from note_include.elem.Ring  import Ring
from note_include.elem.RLWE  import RLWE
from note_include.elem.RLWEp import RLWEp
from note_include.elem.RGSW  import RGSW
from note_include.utils.gadget_decomposition import gadget_decomposition, gadget_composition, format_ring_list

---
Parameters

In [2]:
n   = 16
q   = 7681
B   = 4
std = 3.2

---
Encryption and Homomorphic mulplication ($\textsf{RLWE} \times \textsf{RGSW}$)

In [3]:
rlwe  = RLWE (n, q, std)
rlwep = RLWEp(n, q, std, B)
rgsw  = RGSW (n, q, std, B)

sk, pk          = rlwe.keygen()
coeffs          = [i for i in range(n)]
big_coeffs      = [30 for _ in range(n)]
poly            = Ring(n, q, coeffs)
poly_multiplier = Ring(n, q, big_coeffs)

rgsw_ctxt = rgsw.encrypt(poly_multiplier, sk)
rlwe_ctxt = rlwe.encrypt(poly, sk)
ctxt_mult = rgsw.mult_rlwe(rlwe_ctxt, rgsw_ctxt)

ptxt      = rlwe.decrypt(ctxt_mult, sk)

print("Multiplier              : ", poly_multiplier)
print("Message                 : ", poly)
print("Ideal Result            : ", poly * poly_multiplier)
# print("Plaintext(RLWEp)        : ", ptxt_rlwep)
print("Plaintext(RLWE x RGSW)  : ", ptxt)

Multiplier              :  R(n=16, q=7681, coeffs= 30 + 30x + 30x^2 + 30x^3 + 30x^4 + 30x^5 + 30x^6 + 30x^7 + 30x^8 + 30x^9 + 30x^10 + 30x^11 + 30x^12 + 30x^13 + 30x^14 + 30x^15)
Message                 :  R(n=16, q=7681, coeffs= 0 + 1x + 2x^2 + 3x^3 + 4x^4 + 5x^5 + 6x^6 + 7x^7 + 8x^8 + 9x^9 + 10x^10 + 11x^11 + 12x^12 + 13x^13 + 14x^14 + 15x^15)
Ideal Result            :  R(n=16, q=7681, coeffs= 4081.0 + 4141.0x + 4261.0x^2 + 4441.0x^3 + 4681.0x^4 + 4981.0x^5 + 5341.0x^6 + 5761.0x^7 + 6241.0x^8 + 6781.0x^9 + 7381.0x^10 + 360.0x^11 + 1080.0x^12 + 1860.0x^13 + 2700.0x^14 + 3600.0x^15)
Plaintext(RLWE x RGSW)  :  R(n=16, q=7681, coeffs= 4774.0 + 4695.0x + 4621.0x^2 + 4412.0x^3 + 5081.0x^4 + 5161.0x^5 + 5322.0x^6 + 5663.0x^7 + 6097.0x^8 + 6474.0x^9 + 7003.0x^10 + 7315.0x^11 + 94.0x^12 + 1156.0x^13 + 2090.0x^14 + 2857.0x^15)
